In [ ]:
# uncomment next line
#!python3 -m pip install -r requirements.txt``

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from main import run_all_tests

In [ ]:
tests_results = run_all_tests()

In [ ]:
tests_results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
sns.set_palette('rainbow')

def plot_perm_one(boot_delta, ci, delta_point, perm_null, title='permutation', kde=True):
    boot_delta = np.asarray(boot_delta)
    perm_null  = np.asarray(perm_null)
    lo, hi = ci

    bins = np.histogram_bin_edges(np.r_[boot_delta, perm_null], bins='auto')

    p_two = ((np.abs(perm_null) >= abs(delta_point)).sum() + 1) / (len(perm_null) + 1)

    fig, ax = plt.subplots()
    sns.histplot(boot_delta, bins=bins, stat='density', kde=kde, alpha=0.45,
                 ax=ax, label='bootstrap')

    sns.histplot(perm_null, bins=bins, stat='density', element='step', fill=False, lw=2,
                 ax=ax, label='perm null', color='red')

    ax.axvline(lo, ls='--', lw=2, label=f'CI low {lo:.3g}')
    ax.axvline(hi, ls='--', lw=2, label=f'CI high {hi:.3g}')
    ax.axvline(0.0, ls=':',  lw=2, label='null 0')
    ax.axvline(delta_point, ls='-', lw=2, label=f'point {delta_point:.3g}')

    ax.set(xlabel='mean(B) - mean(A)', ylabel='density',
           title=f'{title}  |  p≈{p_two:.4g}')
    ax.legend(loc='best')
    fig.tight_layout()
    return fig, ax

def plot_mw_one(boot_a12, ci, a12_hat, title='bootstrap A12'):
    boot = np.asarray(boot_a12)
    lo, hi = ci

    fig, ax = plt.subplots()
    sns.histplot(boot, stat='density', bins='auto', kde=True, alpha=0.6, ax=ax)
    ax.axvline(lo, ls='--', lw=2, label=f'CI low {lo:.3g}')
    ax.axvline(hi, ls='--', lw=2, label=f'CI high {hi:.3g}')
    ax.axvline(0.5, ls=':',  lw=2, label='null 0.5')
    ax.axvline(a12_hat, ls='-', lw=2, label=f'point {a12_hat:.3g}')
    ax.set(xlabel='A12', ylabel='density', title=title)
    ax.legend(loc='best')

    ax2 = ax.twinx()
    sns.ecdfplot(boot, ax=ax2)
    ax2.set_ylabel('ecdf')

    fig.tight_layout()
    return fig, ax

def plot_boostrap(boot_delta, ci, delta_hat, title='bootstrap'):
    boot = np.asanyarray(boot_delta)
    lo, hi = ci
    
    fig, ax = plt.subplots()
    sns.histplot(boot, stat='density', bins='auto', kde=True, alpha=0.6, ax=ax)
    ax.axvline(lo, ls='--', lw=2, label=f'CI low {lo:.3g}')
    ax.axvline(hi, ls='--', lw=2, label=f'CI high {hi:.3g}')
    ax.axvline(delta_hat, ls=':',  lw=2, label=f'null {delta_hat}')
    ax.axvline(delta_hat, ls='-', lw=2, label=f'point {delta_hat:.3g}')
    ax.set(xlabel='delta', ylabel='density', title=title)
    ax.legend(loc='best')

    fig.tight_layout()
    return fig, ax

In [ ]:
for experiment, metric_test_results in tests_results.items():
    for metric, test_results in metric_test_results.items():
        for test_result in test_results:
            if test_result.test_name == 'mannwhithney':
                plot_mw_one(
                    test_result.vis_info['resample_distribution'], test_result.ci, test_result.vis_info['a_12'],
                    title=f'bootstrap of a12; {test_result.experiment}-{test_result.metric}-{test_result.test_name}'
                )
            elif test_result.test_name == 'bootstrap_test':
                plot_boostrap(
                    test_result.vis_info['resample_distribution'], test_result.ci, test_result.vis_info['delta_hat'],
                    title=f'bootstrap test; {test_result.experiment}-{test_result.metric}-{test_result.test_name}'
                )
            elif test_result.test_name == 'ttest':
                pass
            else:
                print(f'{experiment}-{metric}-{test_result.test_name}')
                print(test_result.vis_info['resample_distribution'])
                print(test_result.vis_info['delta_hat'])
                print(test_result.vis_info.get('perm_null'))
                plot_perm_one(
                    test_result.vis_info['resample_distribution'], test_result.ci,
                    test_result.vis_info['delta_hat'], test_result.vis_info['perm_null'],
                    title=f'permutation bootstrap; {test_result.experiment}-{test_result.metric}-{test_result.test_name}',
                    kde=True,
                )
